In [ ]:
import dotenv
import matplotlib.pyplot as plt
import neptune
import pandas as pd
import plotly.express as px
import seaborn as sns
import tqdm

dotenv.load_dotenv()
sns.set_theme(style="whitegrid")

In [ ]:
import logging

logging.getLogger("neptune").setLevel(logging.WARNING)

In [ ]:
COL_RENAME = {
    "sys/description": "desc",
    "sys/hostname": "host",
    "sys/id": "id",
    "sys/running_time": "runtime",
    "sys/size": "size",
    "sys/tags": "tags",
    "best_epoch": "best_ep",
    "training/epoch": "epoch",
    "training/hyperparams/artifact_detector": "hp_artifact_det",
    "training/hyperparams/combiner_kwargs/binarize": "hp_comb_bin",
    "training/hyperparams/combiner_kwargs/depth": "hp_comb_depth",
    "training/hyperparams/combiner_kwargs/kernel_size": "hp_comb_ks",
    "training/hyperparams/combiner_kwargs/kind": "hp_comb_kind",
    "training/hyperparams/combiner_kwargs/out_channels": "hp_comb_out_ch",
    "training/hyperparams/combiner_kwargs/soft_factor": "hp_comb_sf",
    "training/hyperparams/combiner_kwargs/yuv_interpolation": "hp_comb_interp",
    "training/hyperparams/combiner_model": "hp_comb_model",
    "training/hyperparams/data_dir": "hp_data_dir",
    "training/hyperparams/detector_kwargs/activation_func": "hp_det_act",
    "training/hyperparams/detector_kwargs/block_depth": "hp_det_depth",
    "training/hyperparams/detector_kwargs/kernel_size": "hp_det_ks",
    "training/hyperparams/detector_kwargs/n_blocks": "hp_det_blocks",
    "training/hyperparams/detector_kwargs/with_ag": "hp_det_ag",
    "training/hyperparams/detector_kwargs/with_fft": "hp_det_fft",
    "training/hyperparams/detector_model": "hp_det_model",
    "training/hyperparams/frac_used": "hp_frac",
    "training/hyperparams/glare_aug_prob": "hp_glare_aug",
    "training/hyperparams/grayscale_patch_prob": "hp_gray_patch",
    "training/hyperparams/img_glob": "hp_img_glob",
    "training/hyperparams/inpainter_kwargs/activation_func": "hp_inp_act",
    "training/hyperparams/inpainter_kwargs/block_depth": "hp_inp_depth",
    "training/hyperparams/inpainter_kwargs/kernel_size": "hp_inp_ks",
    "training/hyperparams/inpainter_kwargs/n_blocks": "hp_inp_blocks",
    "training/hyperparams/inpainter_kwargs/with_ag": "hp_inp_ag",
    "training/hyperparams/inpainter_kwargs/with_fft": "hp_inp_fft",
    "training/hyperparams/inpainter_model": "hp_inp_model",
    "training/hyperparams/learning_rate": "hp_lr",
    "training/hyperparams/mask_blur_factor": "hp_blur",
    "training/hyperparams/max_epochs": "hp_max_ep",
    "training/hyperparams/non_mask_penalty": "hp_non_mask_pen",
    "training/hyperparams/p_flare": "hp_p_flare",
    "training/hyperparams/p_glare": "hp_p_glare",
    "training/hyperparams/p_hq_flare": "hp_p_hq_flare",
    "training/hyperparams/p_overlit": "hp_p_overlit",
    "training/hyperparams/p_sun": "hp_p_sun",
    "training/hyperparams/run_idx": "hp_run_idx",
    "training/hyperparams/run_tags": "hp_run_tags",
    "training/hyperparams/test_dir": "hp_test_dir",
    "training/hyperparams/test_size": "hp_test_size",
    "training/hyperparams/train_dir": "hp_train_dir",
    "training/hyperparams/train_size": "hp_train_size",
    "training/hyperparams/val_dir": "hp_val_dir",
    "training/hyperparams/val_size": "hp_val_size",
    "training/test_detection_loss": "test_det_loss",
    "training/test_inpaint_loss": "test_inp_loss",
    "training/test_mae": "test_mae",
    "training/test_non_mask_edit_loss": "test_non_mask_loss",
    "training/test_ssim": "test_ssim",
    "training/test_total_loss": "test_total_loss",
    "training/test_tv_loss": "test_tv_loss",
    "training/test_vgg_loss": "test_vgg_loss",
    "training/val_detection_loss": "val_det_loss",
    "training/val_inpaint_loss": "val_inp_loss",
    "training/val_mae": "val_mae",
    "training/val_non_mask_edit_loss": "val_non_mask_loss",
    "training/val_ssim": "val_ssim",
    "training/val_total_loss": "val_total_loss",
    "training/val_tv_loss": "val_tv_loss",
    "training/val_vgg_loss": "val_vgg_loss",
}

In [ ]:
ID_THRESHOLD = 428
project = neptune.init_project(mode="read-only")
all_runs_df = (
    project.fetch_runs_table(
        state="inactive",
        columns=COL_RENAME.keys(),
    )
    .to_pandas()
    .rename(columns=COL_RENAME)
)
all_runs_df["id_num"] = all_runs_df.id.str.extract(r"(\d+)$").astype(int)
all_runs_df = all_runs_df[
    (all_runs_df.id_num >= ID_THRESHOLD) & (all_runs_df.epoch == 50)
].reset_index(drop=True)

all_runs_df.head()

In [ ]:
metrics = ["val_total_loss", "val_inpaint_loss", "val_detection_loss"]

run_loss_over_time = []
for rid in tqdm.tqdm(all_runs_df.id.unique()):
    run = neptune.init_run(with_id=rid, mode="read-only")
    metric_vals = []
    for metric in metrics:
        metric_run_df = run[f"training/{metric}"].fetch_values()
        metric_run_df["metric"] = metric
        metric_run_df["id"] = rid
        metric_run_df["epoch"] = metric_run_df.index
        metric_vals.append(metric_run_df)
    run_loss_over_time.append(pd.concat(metric_vals, axis=0))
    run.stop()
run_loss_over_time = pd.concat(run_loss_over_time, axis=0)

In [ ]:
px.line(
    run_loss_over_time[run_loss_over_time.metric == "val_total_loss"],
    x="epoch",
    y="value",
    color="id",
    title="Total Validation Loss",
    labels={"value": "Value", "epoch": "Epoch"},
).show()

In [ ]:
px.line(
    run_loss_over_time[run_loss_over_time.metric == "val_inpaint_loss"],
    x="epoch",
    y="value",
    color="id",
    title="Inpaint Validation Loss",
    labels={"value": "Value", "epoch": "Epoch"},
).show()

In [ ]:
px.line(
    run_loss_over_time[run_loss_over_time.metric == "val_detection_loss"],
    x="epoch",
    y="value",
    color="id",
    title="Detection Validation Loss",
    labels={"value": "Value", "epoch": "Epoch"},
).show()

In [ ]:
px.scatter(
    all_runs_df,
    x="val_det_loss",
    y="val_inp_loss",
    facet_col="hp_non_mask_pen",
    size="hp_train_size",
    color="hp_comb_model",
    hover_name="desc",
    title="Validation Loss vs Train Size",
    labels={
        "val_det_loss": "Validation Detection Loss",
        "val_inp_loss": "Validation Inpainting Loss",
        "hp_train_size": "Train Size",
        "test_det_loss": "Test Detection Loss",
        "test_rec_loss": "Test Reconstruction Loss",
    },
    hover_data=["id"],
).show()